In [1]:
# django shell 환경 설정.
import os
import django
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] ='true'

django.setup()

# Paginator 클래스
- 전체 페이징 처리를 관리하는 클래스
- 전체 데이터관련 정보, 각 페이지당 보여줄 데이터의 정보 등을 제공

# Page 클래스
- 한페이지에대한 데이터를 관리
- Paginator 가 생성해서 제공.
- iterable 타입. 원소로 데이터를 제공.
- Page객체.object_list 속성: 페이지가 가진 데이터들을 List로 반환

In [3]:
from django.core.paginator import Paginator

In [4]:
data_list = list('0123456789abcdefghijklmnopqrstuvwxxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
print(len(data_list))
data_list[:5]
#각 문자가 데이터 한개 (게시판 -> 한개의 게시물) 이라고 가정.

63


['0', '1', '2', '3', '4']

In [5]:
# Paginator생성(데이터들, 한페이지에 보여줄 데이터 개수) # 데이터들: Model.objects.all()
pn = Paginator(data_list, 5)

In [6]:
print("전체데이터수:" , pn.count)

전체데이터수: 63


In [7]:
print("총페이지수:", pn.num_pages)

총페이지수: 13


In [8]:
print("시작~끝 페이지의 범위:", pn.page_range)  # end포함 안되니까 14까지 나옴.. range잖아  // range(1, pn.num_page+1)

시작~끝 페이지의 범위: range(1, 14)


In [9]:
# page 번호를 출력
for p in pn.page_range:
    print(p, end= '\t')

1	2	3	4	5	6	7	8	9	10	11	12	13	

In [12]:
# 각 페이지에 속한 데이터를 조회

page1 = pn.page(1) # pn.page(페이지번호) : 해당 번호 페이지의 데이터들을 Page객체로 반환.
type(page1), page1

(django.core.paginator.Page, <Page 1 of 13>)

In [13]:
page10 = pn.page(10)
page10

<Page 10 of 13>

In [14]:
pn.page(100) # 없는 페이지 조회 exception 발생 -->emptyPage

EmptyPage: 해당 페이지에 결과가 없습니다.

In [15]:
for data in page1:
    print(data)

0
1
2
3
4


In [16]:
for data in page10:
    print(data)

I
J
K
L
M


In [19]:
# 페이지가 가진 데이터를 list로 반환
list_1 = page1.object_list
print(list_1)
list(page1), tuple(page1)

['0', '1', '2', '3', '4']


(['0', '1', '2', '3', '4'], ('0', '1', '2', '3', '4'))

def list(request):
    pageNo = request.GET['page']
    pn = Paginator(Model.objects.all(), 10)
    page = pn.get(pagNo)
    
    return render(request, 'template경로', {'data':page})

## 이전/다음페이지 유무 
- 1페이지 : 이전 페이지 없음, 다음페이지 있음.
- 13페이지: 이전 페이지 있음, 다음페이지 없음.
- 10페이지: 이전,다음 페이지 다있음.

In [20]:
#1 페이지 조회
print('이전페이지 유무:', page1.has_previous())
print('다음페이지 유무:', page1.has_next())

이전페이지 유무: False
다음페이지 유무: True


In [21]:
#10 페이지 조회
print('이전페이지 유무:', page10.has_previous())
print('다음페이지 유무:', page10.has_next())

이전페이지 유무: True
다음페이지 유무: True


In [22]:
# 13 페이지 조회
page13 = pn.page(13)
print('이전페이지 유무:', page13.has_previous())
print('다음페이지 유무:', page13.has_next())

이전페이지 유무: True
다음페이지 유무: False


## 이전/다음페이지 번호 조회

In [72]:
page1.number  # 

1

In [24]:
#page1의 이전 / 다음 페이지 번호
#print('이전페이지번호:', page1.previous_page_number()) #없으면 emptyPage예외 발생
print('다음페이지번호:', page1.next_page_number())

다음페이지번호: 2


In [25]:
# page10의 이전/다음페이지 번홈
print('이전페이지번호:', page10.previous_page_number()) 
print('다음페이지번호:', page10.next_page_number())

이전페이지번호: 9
다음페이지번호: 11


In [27]:
# page13의 이전/다음페이지 번홈
print('이전페이지번호:', page13.previous_page_number()) 
#print('다음페이지번호:', page13.next_page_number()) #없으면 empty page예외 발생

이전페이지번호: 12


## 각 페이지별 데이터 출력(조회해서 출력)


In [35]:
pn = Paginator(data_list, 10) #페이지당 10개 데이터 출력
for pageNO in pn.page_range:
    print(f"{pageNO}페이지")
    for data in pn.page(pageNO):
        print(data, end=', ')
    print("\n=============================================")

1페이지
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
2페이지
a, b, c, d, e, f, g, h, i, j, 
3페이지
k, l, m, n, o, p, q, r, s, t, 
4페이지
u, v, w, x, x, y, z, A, B, C, 
5페이지
D, E, F, G, H, I, J, K, L, M, 
6페이지
N, O, P, Q, R, S, T, U, V, W, 
7페이지
X, Y, Z, 


## 현재 페이지(요청페이지)가 속한 page그룹의 (page_range)에서의 시작 index와 끝 index를 조회

In [37]:
pn = Paginator(data_list,5)
r=pn.page_range

In [40]:
r

range(1, 14)

In [46]:
list(r[0:3]) # 현재페이지 :1,2,3
list(r[3:6]) # 페이지그룸 : 4,5,6// 현재 페이지 :5
list(r[6:9]) # 현재 페이지 9 
## index = page-1 를 이용 슬라이스 해서 페이지 그룹을 구한다. 

[7, 8, 9]

In [65]:
current_page = 11# 현재 페이지
page_group_count = 10 # 한페이 그룹의 페이지 수// 페이지 10개가 1개그룹

# 현재 page가 속함 group의 시작 페이지의 index
start_index = int((current_page-1)/page_group_count) *page_group_count
# 현재 page가 속한 group의 마지막 페이지의 intex
end_index = start_index + page_group_count

#print(start_index, end_index) # 시작페이지/ 끝페이지 번호
r = pn.page_range[start_index:end_index]

for a in r:
    print(a,end=",")

11,12,13,

In [53]:
# current_page = 12
# # n.0 ~ n.9 -> int하면 n 만 남음
# int((current_page-1)/page_group_count) *page_group_count

10

## post 데이터 추가 

In [67]:
from board.models import Category,Post

In [ ]:
c =Category(pk = 1)

In [68]:
for c in Category.objects.all():
    print(c.pk)

1
2
3
4
5


In [69]:
for i in range(350):
    post = Post(title=f'제목-{i}', content=f'내용\n내용\n내용-{i}', category = c)
#   insert,update :모델.save()
    post.save()

In [71]:
Post.objects.all().count()

353